In [1]:
import sys
import cv2
import numpy as np
import torch
from facenet_pytorch import MTCNN, InceptionResnetV1
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QLabel, QVBoxLayout, QFileDialog
from PyQt5.QtGui import QPixmap, QImage
from PyQt5.QtCore import Qt
from facenet_pytorch import MTCNN, InceptionResnetV1
from scipy.spatial.distance import cosine

# Load stored face embeddings
embeddings_path = "face_embeddings.npy"
face_embeddings = np.load(embeddings_path, allow_pickle=True).item()

# Initialize FaceNet and MTCNN models
device = 'cuda' if cv2.cuda.getCudaEnabledDeviceCount() > 0 else 'cpu'
mtcnn = MTCNN(keep_all=False, device=device)
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

class FaceRecognitionApp(QWidget):
    def __init__(self):
        super().__init__()

        # UI setup
        self.setWindowTitle("Face Recognition Attendance System")
        self.setGeometry(100, 100, 600, 400)

        self.layout = QVBoxLayout()

        self.load_image_button = QPushButton("Load Image")
        self.load_image_button.clicked.connect(self.load_image)
        self.layout.addWidget(self.load_image_button)

        self.image_label = QLabel("No image loaded")
        self.image_label.setAlignment(Qt.AlignCenter)
        self.layout.addWidget(self.image_label)

        self.result_label = QLabel("Result: None")
        self.result_label.setAlignment(Qt.AlignCenter)
        self.layout.addWidget(self.result_label)

        self.setLayout(self.layout)

    def load_image(self):
        # Open file dialog to load an image
        file_name, _ = QFileDialog.getOpenFileName(self, "Select Image", "", "Images (*.png *.jpg *.jpeg)")
        if file_name:
            self.process_image(file_name)

    def process_image(self, file_path):
        # Read image using OpenCV
        img = cv2.imread(file_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Display loaded image on UI
        height, width, channel = img_rgb.shape
        bytes_per_line = channel * width
        q_img = QImage(img_rgb.data, width, height, bytes_per_line, QImage.Format_RGB888)
        self.image_label.setPixmap(QPixmap.fromImage(q_img).scaled(300, 300, Qt.KeepAspectRatio))

        # Detect face and extract embeddings
        face = mtcnn(img_rgb)
        if face is not None:
            with torch.no_grad():
                embedding = resnet(face.unsqueeze(0).to(device)).cpu().numpy()[0]

            matched_id, min_distance = self.match_face(embedding)
            if min_distance < 0.8:
                self.result_label.setText(f"Result: Recognized as {matched_id} (distance: {min_distance:.2f})")
            else:
                self.result_label.setText("Result: Unknown person")
        else:
            self.result_label.setText("Result: No face detected")

    def match_face(self, embedding):
        # Compare embedding with stored embeddings to find best match
        min_distance = float('inf')
        matched_id = None
        for person_id, stored_embedding in face_embeddings.items():
            distance = cosine(embedding, stored_embedding)
            if distance < min_distance:
                min_distance = distance
                matched_id = person_id
        return matched_id, min_distance

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = FaceRecognitionApp()
    window.show()
    sys.exit(app.exec_())

C:\Users\25247\anaconda3\envs\facenet_env\lib\site-packages\facenet_pytorch\models\mtcnn.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(state_d

SystemExit: 0

C:\Users\25247\anaconda3\envs\facenet_env\lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
